In [1]:
import numpy as np
import numpy.polynomial.legendre as leg
import scipy as sc
from scipy.stats import norm
from scipy import optimize as op
import math
import cmath
import matplotlib.pyplot as plt
import pandas as pd

# Simulation of stochastic volatility models

This notebook is a companion to the lecture given as part of the 11th Summer School in Mathematical Finance on the 21st of February 2018 at the African Insitute for Mathematical Sciences. In several exercises in this notebook you will learn the pitfalls of simulating stochastic volatility models, using basic Euler schemes.

## Black-Scholes model

Before starting out with stochastic volatility models, we will introduce some basic routines that can be used for simulation. We will focus here on the Black-Scholes model, for which we obviously have a closed-form solution. The closed-form (forward, i.e. undiscounted) option price is implemented as forward_opt below.

In [2]:
class BlackScholes:
    def __init__(self, mu, vol):
        self.mu = mu
        self.vol = vol

    def forward_opt(self, forward, strike, maturity):
        d1 = (math.log(forward / strike) + 0.5 * self.vol * self.vol * maturity) / (self.vol * math.sqrt(maturity))
        d2 = d1 - self.vol * math.sqrt(maturity)
        nd1 = norm.cdf(d1)
        nd2 = norm.cdf(d2)
        return forward * nd1 - strike * nd2
        
    def _simulate_increment(self, dt, randn):
        return (self.mu - 0.5 * self.vol * self.vol) * dt + self.vol * math.sqrt(dt) * randn
    
    def simulate_stock(self, spot, maturity, num_steps):
        dt = maturity / num_steps
        increment = sum(map(lambda x: self._simulate_increment(dt, sc.randn()), range(num_steps)))
        return spot * math.exp(increment)

The next routine prices a call option, taking the model (so far we only have Black-Scholes available), strike, maturity, number of steps and the number of paths as input. It will output the sample mean and the standard deviation of the sample mean.

In [3]:
def price_call_option(model, spot, strike, maturity, num_steps, num_paths):
    realisations = list(map(lambda x: max(model.simulate_stock(spot, maturity, num_steps) - strike, 0.0),
                            range(num_paths)))
    return np.average(realisations), 1.0 / math.sqrt(num_paths) * np.std(realisations)

**Q:** Please test that this works well for the Black-Scholes model, using the closed-form solution.

## Heston model

In the same structure as above we have added a very basic class for simulating the Heston model.
The equations for the Heston model are the following:

$$S(t)=\mu S(t) dt + \sqrt{V(t)} S(t) dW_S(t)$$
$$V(t)=-\kappa (V(t) - \theta) dt + \omega \sqrt{V(t)} dW_V(t)$$

In [4]:
class Heston:
    def __init__(self, mu, kappa, theta, omega, rho, initial_variance):
        self.mu = mu
        self.kappa = kappa
        self.theta = theta
        self.omega = omega
        self.rho = rho
        self.initial_variance = initial_variance
    
    def _simulate_variance_next_step(self, var_prev, dt, randn):
        tmp = self.kappa * dt
        variance = (1.0 - tmp) * var_prev + tmp * self.theta + self.omega * math.sqrt(var_prev * dt) * randn
        return variance
    
    def _simulate_log_asset_next_step(self, log_asset_prev, var_prev, dt, randn):
        increment = (self.mu - 0.5 * var_prev) * dt + math.sqrt(var_prev * dt) * randn
        return log_asset_prev + increment
    
    def simulate_stock(self, spot, maturity, num_steps):
        dt = maturity / num_steps
        var_prev = self.initial_variance
        log_asset_prev = math.log(spot)
        for i in range(num_steps):
            randn_vol = sc.randn()
            randn_spot = self.rho * randn_vol + math.sqrt(1.0 - self.rho * self.rho) * sc.randn()
            var_next = self._simulate_variance_next_step(var_prev, dt, randn_vol)
            log_asset_next = self._simulate_log_asset_next_step(log_asset_prev, var_prev, dt, randn_spot)
            log_asset_prev = log_asset_next
            var_prev = var_next
        return math.exp(log_asset_next)      

The class follows the same structure as the Black-Scholes class, it has a simulate_stock method which produces the value of the stock at maturity. Therefore we can use this class in the price_call_option function we defined earlier.

**Q:** First of all please test that the above implementation reproduces the results for the Black-Scholes model.

**Q:** Second of all, start working on the following example:

$$\kappa = 2, \omega = 1, \rho = -0.3, \theta = 0.09, V(0) = 0.09$$

and price a 5y call option struck at 100, where the initial spot is 100 and the drift is 5% per year. Does the above implementation work for this example? Why not?
Can you measure in what percentage of the paths there is a problem?

**Q:** Please adapt the implementation to get it to work - you can use one of the schemes initially considered in the slides - i.e. absorption or reflection. The benchmark price for this option is 44.94063.

**Q:** Once you have got it to work - try to adapt your implementation to use the full truncation scheme and compare it to the scheme you used previously. How does it perform in comparison?

**Q:** Can you measure the "leaking correlation" problem in this Euler discretisation? How far is the correlation between the log-asset increment and the variance increment from what it should be?